In [5]:
import torch
import deepwave
import numpy as np
from omegaconf import OmegaConf
from scipy.ndimage import gaussian_filter

# 设置config文件路径

In [6]:
config_path = "../config/template.yaml"
cfg = OmegaConf.load(config_path)

In [7]:
#==========================二次设置部分================================================
n_shots = 34
src_depth = 2
first_src =  0
d_src = 20

n_recs = 340
rec_depth = 4
first_rec = 0
d_rcv = 2
#============================================================================
src_loc = np.zeros([n_shots,1,2])
src_loc[:,0,0] = np.arange(n_shots) * d_src + first_src
src_loc[:,0,1] = src_depth
rec_loc = np.zeros([n_shots,n_recs,2])
rec_loc[:,:,0] = np.tile((np.arange(n_recs)*d_rcv+first_rec),(n_shots,1))
rec_loc[:,:,1]=rec_depth
src_loc_path = cfg.forward_par.src_loc_path
rec_loc_path = cfg.forward_par.rec_loc_path
np.save(src_loc_path,src_loc)
np.save(rec_loc_path,rec_loc)
src_loc = torch.from_numpy(src_loc).float()
rec_loc = torch.from_numpy(rec_loc).float()


# 设置地震子波函数

In [8]:
#===================================二次设置部分================================================
freq = 10
nt = 2048
peak_time = 1.5 / freq
dt = cfg.forward_par.dt

#=============================================================================================
wavelet = np.array(deepwave.wavelets.ricker(freq, nt, dt, peak_time)).reshape(1,1,-1)
wavelet = wavelet.repeat(n_shots,axis=0)
wavelet_path = cfg.forward_par.wavelet_path
np.save(wavelet_path,wavelet)
wavelet = torch.from_numpy(wavelet).float()

/tmp/ipykernel_6191/701918878.py:8: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  wavelet = np.array(deepwave.wavelets.ricker(freq, nt, dt, peak_time)).reshape(1,1,-1)


# 设置初始模型与观测数据

In [9]:
#===========================二次设置部分==========================================
init_smooth = 50
#===============================================================================
obs_signal_path = cfg.generate_par.obs_signal_path
vp_true_path = cfg.forward_par.vp_true_path
vp_init_path =  cfg.forward_par.vp_init_path
vp_true = np.load(vp_true_path)
vp_init = 1/gaussian_filter(1/vp_true,init_smooth)
obs_signal = deepwave.scalar(torch.from_numpy(vp_true).float(), source_amplitudes=wavelet,source_locations=src_loc,receiver_locations=rec_loc,grid_spacing=cfg.forward_par.grid_spacing,dt=cfg.forward_par.dt,max_vel=cfg.forward_par.max_vel,pml_width=OmegaConf.to_object(cfg.forward_par.pml_width))[-1]

np.save(obs_signal_path,obs_signal)
np.save(vp_init_path,vp_init)
